In [ ]:
from utils import *
import matplotlib.pyplot as plt

img_path = "../img/Outline/"
# Get the folder information
info = load_folder_information(img_path)
# Generate masks
origin = load_image(os.path.join(img_path, info["origin"]))

print(info)
plt.imshow(origin)

In [ ]:
mask_num = len(info["mask_prefix"])
masks = []
for i in range(mask_num):
    mask = np.zeros_like(origin)
    masked_column = origin.shape[1] // (mask_num + 1) * (i + 1)
    mask[:, masked_column:] = 1
    masks.append(mask)

# for line in range(masks[0].shape[0]):
#     for column in range(masks[0].shape[1]):
#         print(masks[0][line][column][0], end=" ")
#     print()
plt.imshow(masks[6]*255)

In [ ]:
# Image files
img_names = os.listdir(img_path)
# MSE
logits = []
for i in range(mask_num):
    mask = masks[i]
    # for line in range(mask.shape[0]):
    #     for column in range(mask.shape[1]):
    #         print(mask[line][column][0], end=" ")
    #     print()
    mask_prefix = info["mask_prefix"][i]
    imgs = [
        load_image(os.path.join(img_path, img_name))
        for img_name in img_names
        if mask_prefix in img_name
    ]
    mse = [image_MSE(origin, img, mask) for img in imgs]
    logits.append(np.mean(mse))
    # for img in imgs:
    #     plt.imshow(img)
    #     plt.show()

print(logits)

In [ ]:
hpd_path = "/nas/Public/data/HPDv2/test/"
save_path = "/home/lyy/Repos/FACE-image/data/"

import os

file_list = []

if not os.path.exists(save_path):
    os.mkdir(save_path)

for file in os.listdir(hpd_path):
    os.symlink(os.path.join(hpd_path, file), os.path.join(save_path, f"image-{int(file.split('.')[0])}." + file.split(".")[-1] if file.split(".")[-1] != "" else "jpg"))

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

SPLIT = 64
WIDTH = 512

hpd_path = "/nas/Public/data/HPDv2/test/"
generate_path = "/nas/Public/experiment_result/FACE-image/"
logits = {}

model_number = 9
for i in range(model_number):
    logits[i] = []

for file in os.listdir(hpd_path):
    subject_img = cv2.imread(os.path.join(hpd_path, file), cv2.IMREAD_COLOR)[:,:,::-1]
    # TODO: scaled original image
    idx = int(file.split('.')[0])
    # plt.imshow(subject_img)
    # plt.show()
    # print(idx)
    sequence = []
    # each mask strip
    for mask_idx in range(SPLIT - 1):
        generated_img = cv2.imread(os.path.join(generate_path, f"image-{idx}_mask-{mask_idx}_00001_.png"), cv2.IMREAD_COLOR)[:,:,::-1]
        # one strip
        columns = WIDTH // SPLIT * (mask_idx + 1)
        token = np.array(generated_img[:, columns:columns + WIDTH // SPLIT], dtype=np.int16)
        ground_truth = np.array(subject_img[:, columns:columns + WIDTH // SPLIT], dtype=np.int16)

        mse = ((token - ground_truth) ** 2).mean()
        sequence.append(mse)
    print(sequence)
    break